# 📊 NVDA Signal Analysis Project
Backtesting, visualization, and forecasting based on trading signals for NVDA.

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv("/content/NVDA_kainos_su_signalais (1).csv")
df.columns = df.columns.str.strip()
df = df.rename(columns={'date': 'Date'})
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date').reset_index(drop=True)
df.head()
    

In [ ]:

plt.figure(figsize=(14,6))
plt.plot(df['Date'], df['close'], label='Close Price', color='black')
buy_signals = df[df['signal'] == 1.0]
plt.scatter(buy_signals['Date'], buy_signals['close'], color='green', label='Buy Signal', marker='^', s=100)
plt.title('NVDA Price with Buy Signals')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.show()
    

In [ ]:

total_signals = len(buy_signals)
avg_price = buy_signals['close'].mean()
start_price = df['close'].iloc[0]
end_price = df['close'].iloc[-1]
return_pct = (end_price - start_price) / start_price * 100

print(f'Total Buy Signals: {total_signals}')
print(f'Average Buy Price: ${avg_price:.2f}')
print(f'Total Return from {df["Date"].iloc[0].date()} to {df["Date"].iloc[-1].date()}: {return_pct:.2f}%')
    

In [ ]:

from sklearn.linear_model import LinearRegression

df['days'] = (df['Date'] - df['Date'].min()).dt.days
X = df[['days']].values
y = df['close'].values

model = LinearRegression().fit(X, y)

future_days = np.array(range(df['days'].max()+1, df['days'].max()+31)).reshape(-1,1)
future_prices = model.predict(future_days)

plt.figure(figsize=(14,6))
plt.plot(df['Date'], df['close'], label='Historical')
last_date = df['Date'].max()
future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=30)
plt.plot(future_dates, future_prices, label='Forecast', linestyle='--')
plt.title('NVDA Forecast (30 days ahead)')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.show()
    